In [1]:
pip install -U Flask-SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask-login

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bcrypt

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install flask_admin

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install flask_user

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, render_template, url_for, request, redirect, session
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from datetime import datetime
from flask_bcrypt import Bcrypt
import pandas as pd



app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'abc'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
     
                     
class Show(db.Model):
    __tablename__ = 'show'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    show = db.Column(db.String(200))
    poster = db.Column(db.String(200))
    intro = db.Column(db.String(200))
    date = db.Column(db.String(200))
    rate = db.Column(db.String(200))
    time = db.Column(db.String(200))
    band = db.Column(db.String(200))
    venue = db.Column(db.String(200))
    url = db.Column(db.String(200))
    artistinfo = db.Column(db.String(200))
    category = db.Column(db.String(200))
    price = db.Column(db.String(200))
    slug = db.Column(db.String(200), nullable = False)
    
    def __repr__(self):
        return '<city %r: %s>' % self.id, self.name   
                     

class Topic(db.Model):
    __tablename__ = 'topic'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(200))
    subtitle = db.Column(db.String(200))
    category = db.Column(db.String(200))
    slug = db.Column(db.String(200), unique=True, nullable=False)
    def __repr__(self):
        return '<Topic %r' % self.title
    
class Artist(db.Model):
    __tablename__ = 'artist'
    id = db.Column(db.Integer, primary_key=True)
    img = db.Column(db.String(200))
    name = db.Column(db.String(200))
    rate = db.Column(db.String(200))
    tag = db.Column(db.String(200))
    hot = db.Column(db.String(200))
    intro = db.Column(db.String(200))
    video = db.Column(db.String(200))
    music = db.Column(db.String(200))
    comment = db.Column(db.String(200))
    category = db.Column(db.String(200))
    slug = db.Column(db.String(200), unique=True, nullable=False)
    def __repr__(self):
        return '<Topic %r' % self.title
    
db.create_all()

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/shows")
def shows():
    sqlalchemyObj = db.engine.execute('select poster, show, band, date, venue, category, slug from show')
    shows = []
    for i in sqlalchemyObj:
        shows.append(i)
    dataset=[]
    dict={}
    for i in shows:
        dict['poster'] = i[0]
        dict['show'] = i[1]
        dict['band'] = i[2]
        dict['date'] = i[3]
        dict['venue'] = i[4]
        dict['category'] = i[5]
        dict['slug'] = i[6]
        dataset.append(dict.copy())
    print(dataset)
    return render_template('shows.html', shows=dataset)


@app.route("/shows/detail<string:slug>")
def showsdetail(slug):
    sqlalchemyObj = db.engine.execute('select * from show')
    show = Show.query.filter_by(slug= slug).first()
    return render_template("showsdetail.html", show = show)


@app.route("/artists", methods=['GET','POST'])
def artists():
    sqlalchemyObj = db.engine.execute('select name, rate, tag, category, img from artist')
    artists = []
    for i in sqlalchemyObj:
        artists.append(i)
    dataset=[]
    dict={}
    for i in artists:
        dict['name'] = i[0]
        dict['rate'] = i[1]
        dict['tag'] = i[2]
        dict['category'] = i[3]
        dict['img'] = i[4]
        dataset.append(dict.copy())
    print(dataset)
    return render_template('artists.html', artists=dataset)

@app.route("/artists/detail<string:slug>", methods=['POST','GET'])
def artistsdetail(slug):
    sqlalchemyObj = db.engine.execute('select * from artist')
    artist = Artist.query.filter_by(slug= slug).first()

    return render_template("artistsdetail.html", artist = artist)

@app.route("/liveforum", methods=['GET','POST'])
def topics():
    sqlalchemyObj = db.engine.execute('select title, subtitle, category from topic')
    topics = []
    for i in sqlalchemyObj:
        topics.append(i)
    dataset=[]
    dict={}
    for i in topics:
        dict['title'] = i[0]
        dict['subtitle'] = i[1]
        dict['category'] = i[2]
        dataset.append(dict.copy())
    print(dataset)
    return render_template('forum.html', topics=dataset)

    
@app.route("/liveforum/discuss/<string:slug>", methods=['POST','GET'])
def discuss(slug):
    sqlalchemyObj = db.engine.execute('select title, subtitle, slug from topic')
    topic = Topic.query.filter_by(slug= slug).first()
    return render_template('discuss.html', topic = topic)



@app.route("/my")
@login_required
def my():
    return render_template("my.html")

@app.route("/signup", methods=["GET", "POST"])
def signup():
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        hashed_password = bcrypt.generate_password_hash(request.form['password'])
        print(username,email,hashed_password)
        new_user = User(username=username,email=email,password=hashed_password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect(url_for('login'))
        except Exception as e:
            print("Exception occurred.",e)
            return '用户名或邮箱已被占用'
    else:
        return render_template('signup.html')


@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        user = User.query.filter_by(email=request.form["email"]).first()
        if user:
            if bcrypt.check_password_hash(user.password, request.form["password"]):
                login_user(user) 
                return redirect(url_for("my"))
 
        return 'Wrong username or password!'
    return render_template("login.html")
    
@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('index'))


@app.route("/postshow")
@login_required
def postshow():
    return render_template("postshow.html")


@app.route("/newbands")
def newbands():
    return render_template("newbands.html")


@app.route("/venues")
def venues():
    return render_template("venues.html")


if __name__ == "__main__":
    app.run('localhost', 9941)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/helenm/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9941/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 10:44:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:44:12] "GET /static/assets/music/Doo%20Doo%20Bird%20-%20Shewillhewill.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [13/Dec/2021 10:44:12] "GET /static/assets/music/春日部的告别%20-%20海妖.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [13/Dec/2021 10:44:13] "GET /static/assets/music/等太阳落山，就一起做梦%20-%20渡洛西汀.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [13/Dec/2021 10:44:13] "GET /static/assets/music/安定%20ll%20-%20飞古鹿海.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [13/Dec/2021 10:44:13] "GET /static/assets/music/Doo%20Doo%20Bird%20-%20Shewillhewill.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [13/Dec/2021

[{'poster': 'https://s2.showstart.com/img/2021/1026/13/30/3e241a69befa47c88cb35346b987ea3d_1200_1600_1340045.0x0.png?imageMogr2/thumbnail/!275x368r/gravity/Center/crop/!275x368', 'show': 'TAKE ME HOME巡演', 'band': 'ColorfulBar 彩文', 'date': '12/19', 'venue': '北京 糖果LIVE三层', 'category': 'indie', 'slug': 'TAKE ME HOME巡演'}, {'poster': 'https://s2.showstart.com/img/2021/0905/12/00/914ec708cd3d4e35bfdc75adc702b476_1200_1600_1010382.0x0.png?imageMogr2/thumbnail/!275x368r/gravity/Center/crop/!275x368', 'show': '与死去爱人共舞巡演', 'band': 'The Molds', 'date': '12/19', 'venue': '北京 疆进酒·OMNI SPACE', 'category': 'indie', 'slug': '与死去爱人共舞巡演'}, {'poster': 'https://s2.showstart.com/img/2021/1207/12/30/e127b52f42374ce18345767ed751bca7_1200_1600_2864679.0x0.png?imageMogr2/thumbnail/!275x368r/gravity/Center/crop/!275x368', 'show': '《简易度日指南》冬巡', 'band': '打倒三明治', 'date': '12/21', 'venue': '北京 疆进酒·OMNI SPACE', 'category': 'rock', 'slug': '《简易度日指南》冬巡'}, {'poster': 'https://s2.showstart.com/img/2021/0809/18/00/340b5e

127.0.0.1 - - [13/Dec/2021 10:44:17] "GET /postshow HTTP/1.1" 302 -
127.0.0.1 - - [13/Dec/2021 10:44:17] "GET /login?next=%2Fpostshow HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:44:28] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [13/Dec/2021 10:44:28] "GET /my HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 10:44:31] "GET /shows HTTP/1.1" 200 -


[{'poster': 'https://s2.showstart.com/img/2021/1026/13/30/3e241a69befa47c88cb35346b987ea3d_1200_1600_1340045.0x0.png?imageMogr2/thumbnail/!275x368r/gravity/Center/crop/!275x368', 'show': 'TAKE ME HOME巡演', 'band': 'ColorfulBar 彩文', 'date': '12/19', 'venue': '北京 糖果LIVE三层', 'category': 'indie', 'slug': 'TAKE ME HOME巡演'}, {'poster': 'https://s2.showstart.com/img/2021/0905/12/00/914ec708cd3d4e35bfdc75adc702b476_1200_1600_1010382.0x0.png?imageMogr2/thumbnail/!275x368r/gravity/Center/crop/!275x368', 'show': '与死去爱人共舞巡演', 'band': 'The Molds', 'date': '12/19', 'venue': '北京 疆进酒·OMNI SPACE', 'category': 'indie', 'slug': '与死去爱人共舞巡演'}, {'poster': 'https://s2.showstart.com/img/2021/1207/12/30/e127b52f42374ce18345767ed751bca7_1200_1600_2864679.0x0.png?imageMogr2/thumbnail/!275x368r/gravity/Center/crop/!275x368', 'show': '《简易度日指南》冬巡', 'band': '打倒三明治', 'date': '12/21', 'venue': '北京 疆进酒·OMNI SPACE', 'category': 'rock', 'slug': '《简易度日指南》冬巡'}, {'poster': 'https://s2.showstart.com/img/2021/0809/18/00/340b5e

127.0.0.1 - - [13/Dec/2021 10:44:33] "GET /postshow HTTP/1.1" 200 -
